Author: Kyle Herbruger
Date: 10/17/2023

Most of the hand tracking code comes from: https://www.section.io/engineering-education/creating-a-hand-tracking-module/

This program tracks the users on camera hands, and saves the data to two .csv files.
Will write hand location as 0 in the case of no hand being detected to keep time scale accurate.

In [41]:
import cv2
import mediapipe as mp
import numpy as np


# Used to convert protobuf message to a dictionary. 
from google.protobuf.json_format import MessageToDict

In [42]:
class handTracker():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5,modelComplexity=1,trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.modelComplex = modelComplexity
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,self.modelComplex,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils    
        
    def handsFinder(self,image,draw=True):
        imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imageRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:

                if draw:
                    self.mpDraw.draw_landmarks(image, handLms, self.mpHands.HAND_CONNECTIONS)
        return image
    
    def positionFinder(self,image, handNo=0, draw=True):
        lmlist = []
        if self.results.multi_hand_landmarks:
            # Draws out pink circle on pinky tip
            Hand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(Hand.landmark):
                h,w,c = image.shape
                cx,cy = int(lm.x*w), int(lm.y*h)
                lmlist.append([id,cx,cy])
            if draw:
                cv2.circle(image,(cx,cy), 15 , (255,0,255), cv2.FILLED)

        return lmlist

In [62]:
def main():
    fileLoc = './LR_CSV/'
    cap = cv2.VideoCapture(0)
    tracker = handTracker()
    
    timer = 0
    lmListLeft_CSV = []
    lmListRight_CSV = []
    
    # loop to repeatedly scan webcam and process video
    while (timer < 50):
        success,image = cap.read()
        image = cv2.flip(image, 1) 
        image = tracker.handsFinder(image)
        # Original imp: lmListLeft = tracker.positionFinder(image,0)

        # Check if hand detected at all
        if tracker.results.multi_handedness:
            if len(tracker.results.multi_handedness) == 2: # Check if both hands detected
                lmListLeft_CSV.append(tracker.positionFinder(image,0))
                lmListRight_CSV.append(tracker.positionFinder(image,1))
                print('LR')
            else: # Determine which hand is detected
                for i in tracker.results.multi_handedness:
                    label = MessageToDict(i)['classification'][0]['label'] 
                    if label == 'Left':  # Left hand 
                        lmListLeft_CSV.append(tracker.positionFinder(image,0))
                        print('L')
                    if label == 'Right': # Right hand
                        lmListRight_CSV.append(tracker.positionFinder(image,0))
                        print('R')
        # End of handedness detection    
        cv2.imshow("Video",image)
        cv2.waitKey(1)
        timer = timer + 1
    
    npArrLeft = np.zeros((len(lmListLeft_CSV), 21, 3))
    npArrRight = np.zeros((len(lmListRight_CSV), 21, 3))
    for ii in range(len(lmListLeft_CSV)):
        for ib in range(len(lmListLeft_CSV[ii])):
            npArrLeft[ii, ib] = np.array(lmListLeft_CSV[ii][ib])

        for ib in range(len(lmListRight_CSV[ii])):
            npArrRight[ii, ib] = np.array(lmListRight_CSV[ii][ib])
            
    npArrLeft = npArrLeft.astype(int)
    npArrRight = npArrRight.astype(int)
    
    np.savetxt('handIDNum.csv', npArrLeft[:,:,0], delimiter=',')
    np.savetxt(f'{fileLoc}LeftX.csv', npArrLeft[:,:,1], delimiter=',', fmt='%d')
    np.savetxt(f'{fileLoc}LeftY.csv', npArrLeft[:,:,2], delimiter=',', fmt='%d')
    
    np.savetxt(f'{fileLoc}RightX.csv', npArrRight[:,:,1], delimiter=',', fmt='%d')
    np.savetxt(f'{fileLoc}RightY.csv', npArrRight[:,:,2], delimiter=',', fmt='%d')

In [63]:
if __name__ == "__main__":
    main()
    print("Done XD")

LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
LR
Done XD
